# 研究数据集应用于微调的先后顺序对模型效果的影响
此笔记本的微调顺序：`base -->  Cos --> CosMixTri`

### 三、使用 LoRA 进行微调（TriviaQA数据集）¶

设备：M40-24G

<u>基座模型：`model tuned with Cos`(模微调型1) </u>

<u>使用数据：`TriviaQA`</u>

In [1]:
!bash lora_finetune_trivia_web.sh

20240726235858
[2024-07-26 23:59:00,750] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  NVIDIA Inference is only supported on Pascal and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible
[2024-07-26 23:59:03,241] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with traini

## 该模型在Trivia数据集上的测试：

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
path = "/hy-tmp/MiniCPM/finetune/output/LoRA/20240726235858/checkpoint-1000"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(
    path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import os
import json

input_file_path = '/hy-tmp/qa/verified-web-dev.json'
output_file_path = os.path.join(path, 'predictions_tri.json')

prompt = "You are a reading comprehension expert and you need to answer the encyclopaedic questions below based on the text given. Note: Try to be as concise as possible. Do not need to answer in complete sentences.\n" \
            "Example:--Text: Omitted here. --Question: Where in England was Dame Judi Dench born? --Your Answer: York\n" \
            "Here is the question you have to answer:\n"

with open(input_file_path, 'r') as file, open(output_file_path, 'w') as json_file:
    data = json.load(file)['Data']
    result_dict = {}
    for item in data:
        id = item['QuestionId']
        Filename = item['EntityPages']
        for x in item['SearchResults']:
            Filename.append(x['Filename'])
        descriptions = "\n".join([f"Description {index+1}: {desc['Description']}" for index, desc in enumerate(item['SearchResults'])])                 
        query =  f"{prompt}"\
                 f"Question: {item['Question']}\n"\
                 f"Text: {descriptions}"
        res = model.chat(tokenizer, query=query, max_length=1024, top_p=0.5)[0]
        result_dict[f'{id}--{Filename[0]}'] = res
    json.dump(result_dict, json_file, indent=4)
    

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [13]:
!python /hy-tmp/triviaqa_evaluation.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


em=0: OCTOPUS SYNDROME ['list of bond girls in octopussy', 'bond 13', 'list of james bond allies in octopussy', 'magda james bond', 'penelope smallbone', 'kamal kahn', 'octopussy', 'list of james bond villains in octopussy', 'vijay james bond', 'jim fanning james bond', 'general orlov', 'kamal khan', 'octopussy character', 'octopussy film', 'octopussy']
em=0: Grease ['my fair lady musical', 'my fair lady', 'my fair lady 2010 film', 'why can t english 3f', 'my fair lady upcoming film', 'my fair lady 2012 film', 'my fair lady 2014 film', 'my fair lady 2015 film', 'i m ordinary man', 'enry iggins', 'my fair lady']
em=0: New York Yankees ['boston braves', 'boston braves disambiguation', 'boston braves']
em=0: Coca-Cola ['wonderbra', 'wonderbra women', 'wonder bra', 'wonderbra']
em=0: Angela ['bowie disambiguation', 'bowie', 'bowie']
em=0: The Waves ['katrina and waves', 'katrina waves', 'katrina waves']
em=0: Hook ['agnet', 'aglet', 'fluglebinder', 'flugelbinder', 'anglets', 'aglet']
em=0:

## 该模型在Cosmos数据集上的测试：

In [1]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/hy-tmp/MiniCPM/finetune/output/LoRA/20240726235858/checkpoint-1000"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(
    path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
)

In [6]:
import json
import csv
import re
import os

input_file_path = '/hy-tmp/cosmosqa-master/data/test.jsonl'
output_file_path = os.path.join(path, 'predictions_cosmosqa_2nd.lst')

# 逐行读取 JSONL 文件
total_lines = sum(1 for line in open(input_file_path, 'r'))
with open(input_file_path, 'r') as file, open(output_file_path, 'w') as lstfile:
    for line in tqdm(file, total=total_lines, desc="Processing"):
        record = json.loads(line)
        input = f"<用户>You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the number of the option needs to be answered and your answer is only one number.\n" \
         f"Here are the questions you have to answer: \n" \
         f"Text: {record['context']}\n" \
         f"Question: {record['question']}\n" \
         f"Option 0: {record['answer0']}\n" \
         f"Option 1: {record['answer1']}\n" \
         f"Option 2: {record['answer2']}\n" \
         f"Option 3: {record['answer3']}\n"
        res, history = model.chat(tokenizer, query=input, max_length=1024, top_p=0.5)
        label = re.sub(r'\D', '', res)
        lstfile.write(f"{label}\n")

Processing: 100%|██████████| 6963/6963 [2:51:35<00:00,  1.48s/it]


## Base模型在Trivia数据集上的测试：

In [1]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import re
import os

# 加载基座模型
path = '/hy-tmp/MiniCPM/finetune/output/results'

MiniCPM_bf16_path = "/hy-tmp/miniCPM-bf16"
MiniCPM_bf16_tokenizer = AutoTokenizer.from_pretrained(MiniCPM_bf16_path)
MiniCPM_bf16_model = AutoModelForCausalLM.from_pretrained(
    MiniCPM_bf16_path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
)

input_file_path = '/hy-tmp/qa/verified-web-dev.json'
output_file_path = os.path.join(path, 'predictions_tri.json')

prompt = "You are a reading comprehension expert and you need to answer the encyclopaedic questions below based on the text given. Note: Try to be as concise as possible. Do not need to answer in complete sentences.\n" \
            "Example:--Text: Omitted here. --Question: Where in England was Dame Judi Dench born? --Your Answer: York\n" \
            "Here is the question you have to answer:\n"

with open(input_file_path, 'r') as file, open(output_file_path, 'w') as json_file:
    data = json.load(file)['Data']
    result_dict = {}
    for item in data:
        id = item['QuestionId']
        Filename = item['EntityPages']
        for x in item['SearchResults']:
            Filename.append(x['Filename'])
        descriptions = "\n".join([f"Description {index+1}: {desc['Description']}" for index, desc in enumerate(item['SearchResults'])])                 
        query =  f"{prompt}"\
                 f"Question: {item['Question']}\n"\
                 f"Text: {descriptions}"
        res = MiniCPM_bf16_model.chat(MiniCPM_bf16_tokenizer, query=query, max_length=1024, top_p=0.5)[0]
        result_dict[f'{id}--{Filename[0]}'] = res
    json.dump(result_dict, json_file, indent=4)

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to

In [2]:
!python /hy-tmp/triviaqa_evaluation.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


em=0: Question: Rita Coolidge sang the title song for which Bond film?
Text: Description 1: ... Rita Coolidge Performing The title track to the JAMES BOND film OCTOPUSSY. Clip from THE VAL DOONICAN MUSIC SHOW 1983 Featuring Rita Coolidge ... HIGH ...
Answer: OCTOPUSSY ['list of bond girls in octopussy', 'bond 13', 'list of james bond allies in octopussy', 'magda james bond', 'penelope smallbone', 'kamal kahn', 'octopussy', 'list of james bond villains in octopussy', 'vijay james bond', 'jim fanning james bond', 'general orlov', 'kamal khan', 'octopussy character', 'octopussy film', 'octopussy']
em=0: Answer: "On the Street Where You Live" ['my fair lady musical', 'my fair lady', 'my fair lady 2010 film', 'why can t english 3f', 'my fair lady upcoming film', 'my fair lady 2012 film', 'my fair lady 2014 film', 'my fair lady 2015 film', 'i m ordinary man', 'enry iggins', 'my fair lady']
em=0: Answer: John Ford ['john ford 1895 1973', 'sean o feeney', 'ford john 1895 1973', 'john ford stat

## 微调模型1（该模型的前身）在Trivia数据集上的测试：

In [3]:
"""
用微调模型1在Trivia数据集上的验证
"""

import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import csv
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


for model in ['20240725165943']:
    path = f"/hy-tmp/MiniCPM/finetune/output/LoRA/{model}/checkpoint-1000"
    tokenizer = AutoTokenizer.from_pretrained(path)
    model = AutoModelForCausalLM.from_pretrained(
        path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
    )
    
    input_file_path = '/hy-tmp/qa/verified-web-dev.json'
    output_file_path = os.path.join(path, 'predictions_tri.json')
    
    prompt = "You are a reading comprehension expert and you need to answer the encyclopaedic questions below based on the text given. Note: Try to be as concise as possible. Do not need to answer in complete sentences.\n" \
                "Example:--Text: Omitted here. --Question: Where in England was Dame Judi Dench born? --Your Answer: York\n" \
                "Here is the question you have to answer:\n"
    
    with open(input_file_path, 'r') as file, open(output_file_path, 'w') as json_file:
        data = json.load(file)['Data']
        result_dict = {}
        for item in data:
            id = item['QuestionId']
            Filename = item['EntityPages']
            for x in item['SearchResults']:
                Filename.append(x['Filename'])
            descriptions = "\n".join([f"Description {index+1}: {desc['Description']}" for index, desc in enumerate(item['SearchResults'])])                 
            query =  f"{prompt}"\
                     f"Question: {item['Question']}\n"\
                     f"Text: {descriptions}"
            res = model.chat(tokenizer, query=query, max_length=1024, top_p=0.5)[0]
            result_dict[f'{id}--{Filename[0]}'] = res
        json.dump(result_dict, json_file, indent=4)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [4]:
!python /hy-tmp/triviaqa_evaluation.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


em=0: Question: Rita Coolidge sang the title song for which Bond film?
Text: Description 1: ... Rita Coolidge Performing The title track to the JAMES BOND film OCTOPUSSY. Clip from THE VAL DOONICAN MUSIC SHOW 1983 Featuring Rita Coolidge ... HIGH ...
Answer: OCTOPUSY ['list of bond girls in octopussy', 'bond 13', 'list of james bond allies in octopussy', 'magda james bond', 'penelope smallbone', 'kamal kahn', 'octopussy', 'list of james bond villains in octopussy', 'vijay james bond', 'jim fanning james bond', 'general orlov', 'kamal khan', 'octopussy character', 'octopussy film', 'octopussy']
em=0: Answer: "On the Street Where You Live" is a song from the musical "Cabaret". ['my fair lady musical', 'my fair lady', 'my fair lady 2010 film', 'why can t english 3f', 'my fair lady upcoming film', 'my fair lady 2012 film', 'my fair lady 2014 film', 'my fair lady 2015 film', 'i m ordinary man', 'enry iggins', 'my fair lady']
em=0: Answer: John Ford ['john ford 1895 1973', 'sean o feeney', '